## Load Data from Allen Brain Institute and Richiardi, et al.

In [1]:
""" Define location of AHBA data.
    AHBA data can be downloaded from the Allen Human Brain Atlas as six zip files.
    The files should then be unzipped into six separate subdirectories to avoid overwriting identically-named data.
    The main and subdirectories used for this project are defined below.
    """

# AHBA-specific data download and unzip location
ahba_source_dir = "/home/mike/ahba_raw"
donors = ["H0351_2001", "H0351_2002", "H0351_1009", "H0351_1012", "H0351_1015", "H0351_1016", ]

# Richiardi, et al. supplemental data download location
richiardi_supplement_dir = "/home/mike/Dropbox/_papers/gene expression/2015 Richiardi - Supplemental data"
richiardi_probe_file = "Richiardi_Data_File_S2 - 16906 probes and genes.xlsx"
richiardi_sample_file = "Richiardi_Data_File_S1 - 1777 samples.xlsx"


In [2]:
""" Read Richiardi's supplemental data to match their samples, probes, and batch_ids. """

import os
import pandas as pd


richiardi_probes = pd.read_excel(os.path.join(richiardi_supplement_dir, richiardi_probe_file))
richiardi_probes = richiardi_probes.set_index('probe_id')
richiardi_probes.index.name = "probe_name"

richiardi_samples = pd.read_excel(os.path.join(richiardi_supplement_dir, richiardi_sample_file))
richiardi_samples = richiardi_samples.set_index('well_id')

print("Loaded up {:,} Richiardi-based probes and {:,} Richiardi-based samples.".format(
    len(richiardi_probes), len(richiardi_samples)
))

# Actual probes (by id) are in the 'probe_id' column of the richiardi_probes dataframe.
# Actual samples are in the 'well_id' column of the richiardi_samples dataframe.


Loaded up 16,906 Richiardi-based probes and 1,777 Richiardi-based samples.


In [3]:
""" View the data to ensure it worked as expected. """

richiardi_samples.sample(5)


,Brain_id,batch_id,slab_num,slab_type,sample_ontology_id,sample_name,tissue_class,coarse_tissue_class,MNI_x,MNI_y,MNI_z,network_name
well_id,,,,,,,,,,,,
158158299,br6_H0351.1016,275-601,11,CX,4098,"supraparietal lobule, Left, superior bank of g...","supraparietal lobule, superior bank of gyrus",supraparietal lobule,-19.0,-55.000000,75.000000,Z_restOfBrain
4692,br2_H0351.2002,275-202,8,CX,4048,"gyrus rectus, Left",gyrus rectus,gyrus rectus,-11.4,24.200001,-16.000000,Z_restOfBrain
149052742,br5_H0351.1015,275-503,15,CX,4186,"cuneus, Left, peristriate","cuneus, peristriate",cuneus,-7.0,-94.000000,22.000000,Z_restOfBrain
126435041,br4_H0351.1012,275-402,12,CX,4120,"precuneus, Left, superior lateral bank of gyrus","precuneus, superior lateral bank of gyrus",precuneus,-13.0,-47.000000,61.000000,Z_restOfBrain
970,br1_H0351.2001,275-111,18,CX,4023,"superior frontal gyrus, Left, medial bank of g...","superior frontal gyrus, medial bank of gyrus",superior frontal gyrus,-1.6,7.700000,59.799999,Salience


In [4]:
""" View the data to ensure it worked as expected. """

richiardi_probes.sample(5)


,Entrez_id,gene_symbol
probe_name,,
A_23_P54340,84465,MEGF11
A_23_P500410,534,ATP6V1G2
A_23_P156861,26575,RGS17
A_24_P50950,283234,CCDC88B
CUST_2112_PI416261804,51550,CINP


In [5]:
""" Load original AHBA data, and only keep what Richiardi, et al. use. """

import os
import pandas as pd


expression_dataframes = []
for donor in donors:
    print("{}:".format(donor))
    
    samples = pd.read_csv(os.path.join(ahba_source_dir, donor, "SampleAnnot.csv"), header=0, index_col=None, )
    probes = pd.read_csv(os.path.join(ahba_source_dir, donor, "Probes.csv"), header=0, index_col=None, )
    expression = pd.read_csv(os.path.join(ahba_source_dir, donor, "MicroarrayExpression.csv"), header=None, index_col=0, )
    expression.columns = samples['well_id']
    expression = expression.set_index(probes['probe_name'])
    
    # Use list comprehension to keep samples in original order of Richiardi's supplemental data.
    richiardi_filtered_samples = [sid for sid in richiardi_samples.index if sid in samples['well_id'].values]
    print("    {:,} of {:,} samples from {} are used by Richiardi, et al.".format(
        len(richiardi_filtered_samples), len(samples), donor
    ))
    
    # Use list comprehension to keep samples in original order of Richiardi's supplemental data.
    richiardi_filtered_probes = [pid for pid in richiardi_probes.index if pid in probes['probe_name'].values]
    print("    {:,} of {:,} probes from {} are used by Richiardi, et al.".format(
        len(richiardi_filtered_probes), len(probes), donor
    ))

    expression_dataframes.append(expression.reindex(richiardi_filtered_probes)[richiardi_filtered_samples])


H0351_2001:
    498 of 946 samples from H0351_2001 are used by Richiardi, et al.
    16,906 of 58,692 probes from H0351_2001 are used by Richiardi, et al.
H0351_2002:
    380 of 893 samples from H0351_2002 are used by Richiardi, et al.
    16,906 of 58,692 probes from H0351_2002 are used by Richiardi, et al.
H0351_1009:
    180 of 363 samples from H0351_1009 are used by Richiardi, et al.
    16,906 of 58,692 probes from H0351_1009 are used by Richiardi, et al.
H0351_1012:
    260 of 529 samples from H0351_1012 are used by Richiardi, et al.
    16,906 of 58,692 probes from H0351_1012 are used by Richiardi, et al.
H0351_1015:
    222 of 470 samples from H0351_1015 are used by Richiardi, et al.
    16,906 of 58,692 probes from H0351_1015 are used by Richiardi, et al.
H0351_1016:
    237 of 501 samples from H0351_1016 are used by Richiardi, et al.
    16,906 of 58,692 probes from H0351_1016 are used by Richiardi, et al.


In [6]:
""" Combine the six loaded and filtered dataframes into one. """

df_expression = pd.concat(expression_dataframes, axis=1)

# This dataframe has been manually spot-checked and matches the matlab 'MA' matrix perfectly, including column and index order.
df_expression


well_id,1042,948,899,956,770,778,989,990,257,225,...,157772910,157772986,157772858,157772920,157772886,157772950,157772852,157772932,157773028,159439091
probe_name,,,,,,,,,,,,,,,,,,,,,
CUST_9126_PI416261804,1.925130,1.594064,1.968078,1.473066,2.546327,2.141562,2.418547,1.716793,3.593777,3.445025,...,3.015430,2.052985,2.002498,4.088330,1.840219,2.410167,1.117704,0.902501,0.797942,1.432922
CUST_50_PI416408490,5.853280,5.353574,5.077084,5.813961,5.039746,4.917824,5.155898,5.870920,5.274693,5.587318,...,6.606729,6.699496,7.048428,7.164440,7.065678,7.519770,7.808938,7.102441,8.065115,6.773484
CUST_15931_PI416261804,1.473066,1.495026,1.476725,1.473066,2.200937,1.969480,1.516309,1.582553,2.351458,1.542178,...,0.794598,0.859825,0.680751,0.721478,0.883189,0.764315,0.844601,0.761098,0.906440,1.207518
A_23_P2920,2.348390,1.473066,2.255434,2.778622,2.108339,1.682849,2.778308,3.289052,1.473066,1.473066,...,2.393612,3.968676,0.680751,3.157454,1.195282,3.310397,4.027611,3.202751,4.658183,2.587269
A_23_P80570,1.540847,1.685963,2.126519,1.509581,2.421255,2.448850,1.689984,1.875324,2.088800,1.874178,...,1.189261,1.772391,1.822837,1.051241,1.236537,1.238444,2.095870,1.822837,1.313364,0.829019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A_32_P155984,2.210043,1.577918,2.015441,1.476725,2.587799,2.447345,1.719555,1.689639,1.926115,1.950562,...,2.086224,2.471708,1.716746,2.167190,2.231188,1.640802,1.628324,1.583964,2.894996,3.384003
A_23_P336060,2.906538,2.831555,4.121892,2.557226,3.273170,2.158682,3.700598,2.859478,1.476725,2.242247,...,3.228937,2.748959,2.551886,3.365303,3.037767,3.332804,1.340553,3.255166,2.989189,4.431273
A_32_P126214,1.476725,1.841728,1.768105,1.476725,2.465004,2.302847,1.965702,1.959058,2.251680,1.618062,...,2.209501,1.291082,0.827165,1.762383,1.118219,1.195357,1.421993,1.717549,1.144360,1.268565


## Adjust data by brain and batch

In [7]:
""" Regress each probe on brain and batch. """

# Categorical variables need to be one-hot encoded, and their order needs to match original matlab code for comparison
richiardi_brain_ids = sorted(richiardi_samples['Brain_id'].unique())
for brain_id in richiardi_brain_ids:
    richiardi_samples['is_brain_' + brain_id] = (richiardi_samples['Brain_id'] == brain_id).astype(int)

richiardi_batch_ids = sorted(richiardi_samples['batch_id'].unique())
for batch_id in richiardi_batch_ids:
    richiardi_samples['is_batch_' + batch_id] = (richiardi_samples['batch_id'] == batch_id).astype(int)

print("Richiardi's samples represent {:,} brains and {:,} batches.".format(
    len(richiardi_brain_ids), len(richiardi_batch_ids)
))


Richiardi's samples represent 6 brains and 45 batches.


In [8]:
""" Run regressions """

import statsmodels.api as sm


residuals_by_probe = []
for probe in df_expression.index:
    # Transform row of expression values for one probe into a single column
    y = df_expression.loc[[probe, ], :].T

    # First, regress actual expression values against brain ids
    # Remove the first brain from x, and treat it as the baseline. 'Other' brains may deviate from it.
    # Then add back a constant column for the y-intercept.
    x_cols = ['is_brain_' + col for col in richiardi_brain_ids[1:]]
    x = sm.add_constant(richiardi_samples[x_cols])
    brain_model = sm.GLM(y, x)
    brain_results = brain_model.fit()

    # Second, regress brain id residuals against batch ids.
    # Remove the first batch from x, and treat it as the baseline. 'Other' batches may deviate from it.
    # Then add back a constant column for the y-intercept.
    x_cols = ['is_batch_' + col for col in richiardi_batch_ids[1:]]
    x = sm.add_constant(richiardi_samples[x_cols])
    batch_model = sm.GLM(brain_results.resid_response, x)
    batch_results = batch_model.fit()

    # And the values we want to keep and use are the residuals after both brain and batch adjustments.
    residuals_by_probe.append(batch_results.resid_response.rename(probe))

# Combine each probe's residuals back into a complete dataframe.
df_residuals = pd.concat(residuals_by_probe, axis=1).T
df_residuals.index.name = "probe_name"

df_residuals


well_id,1042,948,899,956,770,778,989,990,257,225,...,157772910,157772986,157772858,157772920,157772886,157772950,157772852,157772932,157773028,159439091
probe_name,,,,,,,,,,,,,,,,,,,,,
CUST_9126_PI416261804,-0.462261,-0.173913,0.200101,-0.294910,0.270693,-0.134072,0.031156,-0.670598,1.607685,1.458932,...,1.182416,0.219971,0.169484,2.255316,0.007205,0.577153,-0.715310,-0.930513,-1.035072,-1.443290e-15
CUST_50_PI416408490,0.382273,-0.075982,-0.352472,0.384405,-0.134426,-0.256347,-0.315110,0.399913,-0.315876,-0.003251,...,-0.583193,-0.490426,-0.141494,-0.025482,-0.124244,0.329848,0.619016,-0.087482,0.875192,8.881784e-16
CUST_15931_PI416261804,-0.291798,-0.091831,-0.110132,-0.113791,0.338370,0.106912,-0.248555,-0.182311,0.689847,-0.119433,...,-0.081120,-0.015893,-0.194967,-0.154240,0.007471,-0.111403,-0.031117,-0.114620,0.030722,-1.665335e-16
A_23_P2920,0.098576,-0.640421,0.141947,0.665134,-0.048537,-0.474026,0.528495,1.039238,-0.481738,-0.481738,...,-0.341026,1.234037,-2.053887,0.422816,-1.539356,0.575759,1.292973,0.468113,1.923545,-1.776357e-15
A_23_P80570,-0.322954,0.057297,0.497853,-0.119086,0.222112,0.249708,-0.173816,0.011524,0.181704,-0.032919,...,-0.272551,0.310579,0.361025,-0.410571,-0.225274,-0.223367,0.634058,0.361025,-0.148448,-1.665335e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A_32_P155984,0.158224,-0.141640,0.295883,-0.242832,0.242020,0.101566,-0.332264,-0.362181,0.013787,0.038234,...,0.052668,0.438152,-0.316810,0.133634,0.197632,-0.392754,-0.405232,-0.449592,0.861440,1.776357e-15
A_23_P336060,0.187981,-0.049194,1.241143,-0.323523,0.634442,-0.480046,0.982040,0.140921,-0.902792,-0.137270,...,0.358634,-0.121344,-0.318416,0.495001,0.167465,0.462502,-1.529750,0.384863,0.118886,0.000000e+00
A_32_P126214,-0.612450,-0.199939,-0.273562,-0.564942,0.104780,-0.057377,-0.123473,-0.130117,0.239191,-0.394426,...,0.710448,-0.207970,-0.671887,0.263330,-0.380833,-0.303696,-0.077059,0.218496,-0.354692,-1.776357e-15


In [9]:
""" Save data for future use without having to re-process. """

import pandas as pd


# Other dataframes use 'probe_id' rather than 'probe_name' for indexing. Make this dataframe match before saving.
# (any donor's Probes.csv file would work for this)
probes = pd.read_csv(os.path.join(ahba_source_dir, "H0351_2002", "Probes.csv"), header=0, index_col=None, )
name_to_id_map = probes[['probe_name', 'probe_id', ]].set_index('probe_name')['probe_id'].to_dict()

df_residuals.index = df_residuals.index.map(name_to_id_map)
df_residuals.index.name = "probe_id"

# Save residuals.
# This file should contain identical data, but in a different format, as MA_resid.mat
# df_residuals.to_pickle("brain_and_batch_adjusted_residuals.df")
df_residuals.to_csv("brain_and_batch_adjusted_residuals.csv")
